In [1]:
import pandas as pd

df = pd.read_csv("C:/Users/karth/OneDrive/Desktop/retail-forecast-segmentation-inventory/data/processed/superstore_clean.csv")
df.head()


,Order Date,Customer ID,Category,Sub-Category,Product ID,Sales,Quantity,Profit,Year,Month
0,2016-11-08,CG-12520,Furniture,Bookcases,FUR-BO-10001798,261.9600,2,41.9136,2016,2016-11
1,2016-11-08,CG-12520,Furniture,Chairs,FUR-CH-10000454,731.9400,3,219.5820,2016,2016-11
2,2016-06-12,DV-13045,Office Supplies,Labels,OFF-LA-10000240,14.6200,2,6.8714,2016,2016-06
3,2015-10-11,SO-20335,Furniture,Tables,FUR-TA-10000577,957.5775,5,-383.0310,2015,2015-10
4,2015-10-11,SO-20335,Office Supplies,Storage,OFF-ST-10000760,22.3680,2,2.5164,2015,2015-10


In [2]:
product_metrics = df.groupby(['Product ID','Sub-Category','Category']).agg({
    'Quantity': 'sum',
    'Sales': 'sum',
    'Profit': 'sum'
}).reset_index()

product_metrics.rename(columns={
    'Quantity': 'Total_Quantity',
    'Sales': 'Total_Sales',
    'Profit': 'Total_Profit'
}, inplace=True)

product_metrics.head()


,Product ID,Sub-Category,Category,Total_Quantity,Total_Sales,Total_Profit
0,FUR-BO-10000112,Bookcases,Furniture,9,825.174,-117.8820
1,FUR-BO-10000330,Bookcases,Furniture,10,1064.624,24.1960
2,FUR-BO-10000362,Bookcases,Furniture,14,2154.348,311.1836
3,FUR-BO-10000468,Bookcases,Furniture,21,723.842,-133.1092
4,FUR-BO-10000711,Bookcases,Furniture,12,851.760,229.9752


In [3]:
product_metrics = product_metrics.sort_values('Total_Sales', ascending=False)
product_metrics['Cumulative_Sales'] = product_metrics['Total_Sales'].cumsum()
total_sales = product_metrics['Total_Sales'].sum()
product_metrics['Cumulative_Pct'] = product_metrics['Cumulative_Sales'] / total_sales

def abc_class(pct):
    if pct <= 0.8:
        return 'A'  # top ~80% sales
    elif pct <= 0.95:
        return 'B'
    else:
        return 'C'

product_metrics['ABC_Class'] = product_metrics['Cumulative_Pct'].apply(abc_class)

product_metrics.head()


,Product ID,Sub-Category,Category,Total_Quantity,Total_Sales,Total_Profit,Cumulative_Sales,Cumulative_Pct,ABC_Class
1614,TEC-CO-10004722,Copiers,Technology,20,61599.824,2.519993e+04,61599.824,0.026815,A
776,OFF-BI-10003527,Binders,Office Supplies,31,27453.384,7.753039e+03,89053.208,0.038766,A
1642,TEC-MA-10002412,Machines,Technology,6,22638.480,-1.811078e+03,111691.688,0.048621,A
80,FUR-CH-10002024,Chairs,Furniture,39,21870.576,5.684342e-14,133562.264,0.058141,A
691,OFF-BI-10001359,Binders,Office Supplies,37,19823.479,2.233505e+03,153385.743,0.066771,A


In [4]:
product_metrics.to_csv("C:/Users/karth/OneDrive/Desktop/retail-forecast-segmentation-inventory/data/processed/inventory_metrics.csv", index=False)
